In [21]:
import os
import json
import argparse
from random import shuffle, seed
import string
# non-standard dependencies:
import h5py
import numpy as np
import torch
import torchvision.models as models
from torch.autograd import Variable
import skimage.io

In [15]:
import json
imgs = json.load(open('/home/vivek/data/COCO/image/dataset_coco.json', 'r'))

In [16]:
imgs.keys()

dict_keys(['images', 'dataset'])

In [9]:
imgs['images'][0]

{'cocoid': 391895,
 'filename': 'COCO_val2014_000000391895.jpg',
 'filepath': 'val2014',
 'imgid': 0,
 'sentences': [{'imgid': 0,
   'raw': 'A man with a red helmet on a small moped on a dirt road. ',
   'sentid': 770337,
   'tokens': ['a',
    'man',
    'with',
    'a',
    'red',
    'helmet',
    'on',
    'a',
    'small',
    'moped',
    'on',
    'a',
    'dirt',
    'road']},
  {'imgid': 0,
   'raw': 'Man riding a motor bike on a dirt road on the countryside.',
   'sentid': 771687,
   'tokens': ['man',
    'riding',
    'a',
    'motor',
    'bike',
    'on',
    'a',
    'dirt',
    'road',
    'on',
    'the',
    'countryside']},
  {'imgid': 0,
   'raw': 'A man riding on the back of a motorcycle.',
   'sentid': 772707,
   'tokens': ['a',
    'man',
    'riding',
    'on',
    'the',
    'back',
    'of',
    'a',
    'motorcycle']},
  {'imgid': 0,
   'raw': 'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a backgro

In [5]:
len(imgs['images'])

123287

In [7]:
len(imgs['dataset'])

4

In [17]:
imgs = imgs['images']

In [23]:
count_thr = 5
# Count the number of words. : use dict.items(), dict.keys(), and dict.values() respectively.
counts ={}

for img in imgs:
    for sent in img['sentences']:
        for w in sent['tokens']:
            counts[w] = counts.get(w, 0) + 1
cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
cw[:20]

[(1019785, 'a'),
 (224758, 'on'),
 (212689, 'of'),
 (206178, 'the'),
 (191793, 'in'),
 (161216, 'with'),
 (146755, 'and'),
 (102390, 'is'),
 (75957, 'man'),
 (71183, 'to'),
 (55190, 'sitting'),
 (51987, 'an'),
 (50467, 'two'),
 (44506, 'at'),
 (44297, 'standing'),
 (43707, 'people'),
 (42776, 'are'),
 (38867, 'next'),
 (37898, 'white'),
 (35372, 'woman')]

In [50]:
total_words = sum(counts.values())

In [48]:
len(counts)

27929

In [31]:
bad_words = [w for w, n in counts.items() if n <= count_thr]

In [56]:
bad_words[0]

'wreathed'

In [34]:
len(bad_words)

18443

In [35]:
bad_words[:10]

['wreathed',
 'bakeware',
 'inhales',
 'goodness',
 'coworker',
 'dwellers',
 'bmx',
 'geoup',
 'gouged',
 'socket']

In [53]:
bad_count = sum(counts[w] for w in bad_words)

In [55]:
bad_count

32382

In [38]:
vocab = [w for w, n in counts.items() if n > count_thr]

In [39]:
vocab[:10]

['a', 'man', 'with', 'red', 'helmet', 'on', 'small', 'moped', 'dirt', 'road']

In [40]:
len(vocab)

9486

In [47]:
print('number of bad words: %d/%d = %.2f%%'  % (len(bad_words), len(counts), len(bad_words)*100.0/len(counts)))

number of bad words: 18443/27929 = 66.04%


In [57]:
print('number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count*100.0/total_words))

number of UNKs: 32382/6454115 = 0.50%


In [58]:
# lets look at the distribution of lengths as well

sent_length ={}
for img in imgs:
    for sent in img['sentences']:
        txt = sent['tokens']
        nw = len(txt)
        sent_length[nw] = sent_length.get(nw, 0) + 1
        
max_length = max(sent_length.keys())

In [59]:
max_length

49

In [60]:
if bad_count > 0:
    vocab.append('UNK')

In [71]:
for img in imgs:
    img['final_captions'] = []
    for sent in img['sentences']:
        txt = sent['tokens']
        caption = [w if counts.get(w,0) > count_thr else 'UNK' for w in txt]
        img['final_captions'].append(caption)

In [65]:
len(counts)

27929

In [66]:
len(imgs)

123287

In [73]:
vocab[:10]

['a', 'man', 'with', 'red', 'helmet', 'on', 'small', 'moped', 'dirt', 'road']

In [74]:
sum(len(img['final_captions']) for img in imgs)

616767